# Data Querying and Exporting

This notebook demonstrates the process of querying data from Delta Lake and visualizing it in various formats.  It is particularly set up for the Dryad dataset, but the process for accessing other datasets is the same.  

## Restimates


In [12]:
import pandas as pd
import numpy as np

restimates = pd.read_csv("/app/data/files/11_Restimates_ALL_DailyActivity.csv")

def matlab_datenum_to_datetime_vectorized(
    matlab_serial_dates: np.ndarray,
) -> np.ndarray:
    """
    Converts a vector of MATLAB datenum values to pandas datetime in a vectorized manner.

    MATLAB datenum starts from year 0000-01-00, while Python's datetime starts from 1970-01-01.
    We need to adjust by subtracting the number of days between these dates.
    """
    # MATLAB serialized dates start from 0000-01-01
    matlab_start_date = np.datetime64("0000-01-01")

    # Split into days and fractional days
    days = np.floor(matlab_serial_dates).astype(int)
    fraction = matlab_serial_dates - days

    # Convert MATLAB serial date to Python datetime
    converted_dates = (
        matlab_start_date
        + days.astype("timedelta64[D]")
        + (fraction * 24 * 3600).astype("timedelta64[s]")
    ).astype("datetime64[ns]")
    return converted_dates




sample = restimates.head(200)
display(sample)

# display(restimates[restimates["Deployment_ID"] == 2004001])
# # restimates[restimates["Deployment_ID"] == 2004001]

,unique_Days,Days_Elapsed,daily_recording,Lat,Long,Lon360,daily_diving,daily_SI,daily_long_SI,daily_filtered_long_drift,...,TOPPID,SEALID,Percent_of_Trip,dailydive_glide,dailydive_long_glide,dailydive_KAMI,daily_all_sleep,daily_all_REM,dailydive_sleep,dailydive_REM
0,732000,0,14.240000,37.223897,-122.513835,237.486165,12.264444,1.751111,0.0,0.124444,...,2004001,R881,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,732001,1,23.975556,37.616749,-123.104691,236.895309,21.828889,1.731111,0.0,4.800000,...,2004001,R881,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,732002,2,23.895556,37.956043,-123.254476,236.745524,21.957778,1.426667,0.0,2.113333,...,2004001,R881,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,732003,3,23.988889,38.643718,-123.548055,236.451945,22.002222,1.540000,0.0,0.804444,...,2004001,R881,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,732004,4,24.000000,39.051504,-124.127636,235.872364,21.728889,1.813333,0.0,0.997778,...,2004001,R881,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,732017,17,24.000000,44.202867,-133.267593,226.732407,21.180000,2.444444,0.0,0.731111,...,2004005,O176,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,732018,18,24.000000,44.616750,-133.809109,226.190891,21.831111,1.675556,0.0,1.806667,...,2004005,O176,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,732019,19,24.000000,45.013053,-134.148557,225.851443,21.708889,1.840000,0.0,0.231111,...,2004005,O176,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,732020,20,24.000000,45.464614,-134.044286,225.955714,21.560000,1.975556,0.0,0.775556,...,2004005,O176,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
metadata = pd.read_csv("/app/data/files/NES_TrackingDiving_MetaData.csv")
metadata[
    ['Animal_ID', 'Deployment_ID', 'Deployment_Trip']
]

metadata['is2017'] = metadata['Deployment_ID'].apply(lambda x: str(x).startswith('2017'))
metadata['max_QC_flag'] = metadata[['Data_Track_QCFlag', 'Data_TDR1_QCFlag']].max(axis=1)
metadata = metadata[metadata['max_QC_flag'] <= 3]
# metadata = metadata[metadata['is2017']]
metadata[metadata['Deployment_ID'] == 2017001]

,Animal_ID,Deployment_ID,Tags_SatTag_Manufacturer,Tags_SatTag_Model,Tags_SatTag_ID,Tags_PTT,Data_Track_QCFlag,Tags_SatTag_Comments,Tags_TDR1_Manufacturer,Tags_TDR1_Model,...,Deployment_Trip,Animal_BirthYear,Animal_AgeClass,Animal_Sex,Deployment_Manipulation,Deployment_ManipulationType,Animal_HadPup,Deployment_Year,is2017,max_QC_flag
558,6018,2017001,Wildlife Computers,SPOT5,13s1533,139053.0,3,NaN,Wildlife Computers,Mk9,...,PB,2011.0,Adult,F,N,NaN,NaN,2017.0,True,3


In [60]:
restimates_quality = restimates[restimates['TOPPID'].isin(metadata['Deployment_ID'])]
restimates_quality


,unique_Days,Days_Elapsed,daily_recording,Lat,Long,Lon360,daily_diving,daily_SI,daily_long_SI,daily_filtered_long_drift,...,TOPPID,SEALID,Percent_of_Trip,dailydive_glide,dailydive_long_glide,dailydive_KAMI,daily_all_sleep,daily_all_REM,dailydive_sleep,dailydive_REM
0,732000,0,14.240000,37.223897,-122.513835,237.486165,12.264444,1.751111,0.0,0.124444,...,2004001,R881,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,732001,1,23.975556,37.616749,-123.104691,236.895309,21.828889,1.731111,0.0,4.800000,...,2004001,R881,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,732002,2,23.895556,37.956043,-123.254476,236.745524,21.957778,1.426667,0.0,2.113333,...,2004001,R881,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,732003,3,23.988889,38.643718,-123.548055,236.451945,22.002222,1.540000,0.0,0.804444,...,2004001,R881,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,732004,4,24.000000,39.051504,-124.127636,235.872364,21.728889,1.813333,0.0,0.997778,...,2004001,R881,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52739,738174,213,24.000000,37.520102,-125.131445,234.868555,22.451111,1.288889,0.0,2.593333,...,2020024,6609,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52740,738175,214,24.000000,37.133202,-124.512049,235.487951,22.402222,1.328889,0.0,1.797778,...,2020024,6609,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52741,738176,215,24.000000,36.749071,-123.653268,236.346732,21.744444,1.995556,0.0,3.366667,...,2020024,6609,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52742,738177,216,24.000000,36.693417,-122.907452,237.092548,22.200000,1.548889,0.0,3.200000,...,2020024,6609,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN


`daily_filtered_long_drift_long_SI`: sum of time spent sleeping

`daily_filtered_long_drift`: sum of time spent sleeping underwater

`daily_long_SI`: sum of time spent sleeping at surface in hours

`Lon360`: longitude in 360

`Lat`: latitude


In [ ]:
import math
from shapely.geometry import Polygon
from shapely.ops import unary_union
import geopandas as gpd
import contextily as ctx
from shapely.geometry import LineString

def get_mask(gdf, buffer_size=0.1):
    gdf_poly = gdf.geometry.apply(lambda x: x.buffer(buffer_size))
    dissolved_poly = unary_union(gdf_poly.geometry)
    dissolved_gdf = gpd.GeoDataFrame(
        pd.DataFrame({"id": [0]}),
        geometry=[dissolved_poly],
        crs="EPSG:4326"
    )
    return dissolved_gdf


def create_line_gdf(gdf, description="Sick Seal 2017001"):
    points = list(gdf.geometry.apply(lambda pt: (pt.x, pt.y)))
    return gpd.GeoDataFrame(
        pd.DataFrame({"id": [0], "description": [description]}),
        geometry=[LineString(points)],
        crs="EPSG:4326"
    )
    

def plot_restimates(df, buffer_size=0.7, highlight_id=2017001):
    df = df.reset_index()
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(
            df["Lon360"], 
            df["Lat"],
        ),
        crs="EPSG:4326"
    )
    
    dissolved_gdf = get_mask(gdf, buffer_size)
    ax = dissolved_gdf.plot(alpha=0.5)
    gdf[gdf['TOPPID'] == highlight_id].plot(markersize=0.1, ax=ax, legend=True, color='black')
    
    specimen = gdf[gdf['TOPPID'] == highlight_id]
    line_gdf = create_line_gdf(specimen)
    line_gdf.plot(ax=ax, legend=True, color='black')
    
    return


def explore_restimates(df, buffer_size=0.7, highlight_id=2017001):
    
    import folium
    df = df.reset_index()
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(
            df["Lon360"], 
            df["Lat"],
        ),
        crs="EPSG:4326"
    )
    dissolved_gdf = get_mask(gdf, buffer_size)
    
    specimen = gdf[gdf['TOPPID'] == highlight_id]
    line_gdf = create_line_gdf(specimen)
    
    m = folium.Map(location=[0, 0], zoom_start=2)

    # Add the first GeoDataFrame
    folium.GeoJson(
        dissolved_gdf,
        name='Base Distribution',
        style_function=lambda x: {
            'fillColor': 'blue',
            'fillOpacity': 0.5
        },
    ).add_to(m)
    
    folium.GeoJson(
        gdf,
        name='Base Distribution',
        style_function=lambda x: {
            'fillColor': 'blue',
            'fillOpacity': 0.5,
            'alpha': 0.5
        },
    ).add_to(m)

    # # Add the second GeoDataFrame
    folium.GeoJson(
        line_gdf,
        name='2017001',
        style_function=lambda x: {
            'fillColor': 'black',
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.5
        },
        tooltip=folium.GeoJsonTooltip(fields=['description'])
    ).add_to(m)
    
    folium.TileLayer(
        tiles=ctx.providers.Esri.WorldImagery,
        name='Satellite',
        attr=ctx.providers.Esri.WorldImagery['attribution']
    ).add_to(m)

    # # Add layer control to toggle layers
    # folium.LayerControl().add_to(m)
    
    return m

    
explore_restimates(restimates_quality)

## Querying Data

In [ ]:
import importlib
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)

from DiveDB.services.duck_pond import DuckPond
import pandas as pd

duckpond = DuckPond()

df = duckpond.conn.sql(    
    """
    SELECT 
        deployment, 
        datetime,
        date_trunc('day', datetime) as day, 
        value.float as value,
        label
    FROM DataLake 
    WHERE 
        label IN ('lat', 'lon', 'corr_depth')
    """
).df()

df2 = pd.merge(
    pd.merge(
        df[df['label'] == 'lat'].rename(columns={'minval': 'min_lat', 'maxval': 'max_lat'}).drop(columns=["label", "day"]),
        df[df['label'] == 'lon'].rename(columns={'minval': 'min_lon', 'maxval': 'max_lon'}).drop(columns=["label", "day"]),
        on=['deployment', 'datetime'],
        suffixes=('_lat', '_lon')
    ),
    df[df['label'] == 'corr_depth'].rename(columns={'minval': 'min_depth', 'maxval': 'max_depth'}).drop(columns=["label"]),
    on=['animal', 'day'],
    suffixes=('', '_depth')
)


In [ ]:
import geopandas as gpd
import contextily as ctx

gdf = gpd.GeoDataFrame(
    df2.reset_index(),
    geometry=gpd.points_from_xy(df2.reset_index().min_lon, df2.reset_index().min_lat),
    crs="EPSG:4326"
)

gdf = gdf[gdf.min_lon < -120]

ax = gdf.plot(column="animal", figsize=(15, 15), legend=True, alpha=0.7)
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Esri.WorldImagery, attribution_size=3, zoom=4)


In [ ]:
import math
from shapely.geometry import Polygon

FACTOR = 1

gdf = gpd.GeoDataFrame(
    df2.reset_index(),
    geometry=gpd.points_from_xy(df2.reset_index().min_lon.apply(lambda x: math.floor(x * FACTOR) / FACTOR), df2.reset_index().min_lat.apply(lambda x: math.floor(x * FACTOR) / FACTOR)),
    crs="EPSG:4326"
)

gdf = gdf[gdf.min_lon < -120]


display(gdf)

gdf2 = gdf[["geometry", "max_depth"]].groupby(["geometry"]).count().rename(columns={"max_depth": "count"}).sort_values(by="count", ascending=False).reset_index()
display(gdf2)

gdf2.geometry = gdf2.geometry.apply(lambda pt: Polygon([(pt.x, pt.y), (pt.x+1/FACTOR, pt.y), (pt.x+1/FACTOR, pt.y+1/FACTOR), (pt.x, pt.y+1/FACTOR)]))

gdf = gpd.GeoDataFrame(
    gdf2.reset_index(),
    geometry=gdf2.geometry,
    crs="EPSG:4326"
)


ax = gdf.plot(column="count", cmap="YlOrRd", figsize=(15, 15), legend=True, alpha=0.9)
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Esri.WorldImagery, attribution_size=3, zoom=4)


In [ ]:
import math
from shapely.geometry import Polygon

FACTOR = 1

gdf = gpd.GeoDataFrame(
    df2.reset_index(),
    geometry=gpd.points_from_xy(df2.reset_index().min_lon.apply(lambda x: math.floor(x * FACTOR) / FACTOR), df2.reset_index().min_lat.apply(lambda x: math.floor(x * FACTOR) / FACTOR)),
    crs="EPSG:4326"
)

gdf = gdf[gdf.min_lon < -120]


display(gdf)

gdf2 = gdf[["animal", "geometry"]].groupby(["geometry"]).agg({"animal": "nunique"}).rename(columns={"animal": "count"}).sort_values(by="count", ascending=False).reset_index()

display(gdf2)

gdf2.geometry = gdf2.geometry.apply(lambda pt: Polygon([(pt.x, pt.y), (pt.x+1/FACTOR, pt.y), (pt.x+1/FACTOR, pt.y+1/FACTOR), (pt.x, pt.y+1/FACTOR)]))

gdf = gpd.GeoDataFrame(
    gdf2.reset_index(),
    geometry=gdf2.geometry,
    crs="EPSG:4326"
)


ax = gdf.plot(column="count", cmap="YlOrRd", vmax=10, figsize=(15, 15), legend=True, alpha=0.9)
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Esri.WorldImagery, attribution_size=3, zoom=4)


### Using the DuckPond connection to query directly
More complex queries can be run directly on the DuckPond connection. This is useful for queries that may not be supported by the `get_delta_data` method which has those involving grouping or aggregations. 

DuckDB runs sql very similar in syntax to other SQL databases. A full breakdown of the syntax can be found [in the documenation](https://duckdb.org/docs/sql/introduction).

The connection object can be found in the `duckpond.conn` attribute. To run queries, use the `sql` method which also returns a [DuckDB DuckDBPyConnection](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyConnection) which can be used to convert the data in many different formats including the following ([see documentation for a full list](https://duckdb.org/docs/api/python/conversion#result-conversion-duckdb-results-to-python))
- NumPy Array (`.fetchnumpy()`)
- Pandas DataFrame (`.df()`)
- Arrows Table (`.arrow()`)
- Polars DataFrame (`.pl()`)

##### Example:

In [ ]:
import importlib

# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

df = duckpond.conn.sql(f"""
SELECT label, avg(value) as mean_data
FROM (
    SELECT label, value.int as value
    FROM DataLake
    WHERE label = 'sensor_data_ecg'
    OR label = 'sensor_data_light'
)
GROUP BY label
""").df()

display(df)


## Chaining Queries
Queries can be chained together to form a pipeline. This is useful for running complex queries that involve multiple steps.

##### Example:

In [ ]:
import importlib

# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

# Get the filtered data
filtered_data = duckpond.get_delta_data(    
    animal_ids="oror-002",
    frequency=10, # Resample values to 10 Hz and make sure each signal has the same time intervals
    # Aggregation of events (think state events - behaviors) type: state (has state and end dates)
    classes="sensor_data_accelerometer",
    
)

display(filtered_data)


## Query Variables
Sometimes we don't want to hardcode variables in our queries. We can use the `execute` method to pass variables to the query.

##### Example:

In [ ]:
import importlib

# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

label = "sensor_data_temperature"
df = duckpond.conn.execute(f"""
SELECT label, avg(value) as mean_data
FROM (
    SELECT label, value.float as value
    FROM DataLake
    WHERE label = $1
)
GROUP BY label
""", [label]).df()
display(df)

## Query Metadata Database
We can also query the Metadata Database directly. This is useful for querying data that is not stored in Delta Lake and joining it for queries on measurement data.

##### Example:

In [ ]:
import importlib

# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()


# Show all tables we have access to
print(duckpond.get_db_schema())

df = duckpond.conn.sql("""
SELECT value.float as value
FROM DataLake 
JOIN Metadata.public.Animals ON DataLake.animal = Animals.id
WHERE Animals.project_id = 'test12_Wednesday'
AND label = 'sensor_data_temperature'
""").df()


display(df)

## Exporting Data to EDF
When it's easier to work with EDF files, we can export the data to an EDF file. This is useful for working with the data in other software packages.

The `create_mne_edf` function takes a DuckDB connection and a file path and creates an EDF file. 

*Note: it currently requires a lot of memory. Can be improved.*
*Note: it's lacking support for most info fields in the EDF file. Can be improved.*

##### Example:

In [ ]:
import os
import importlib
import DiveDB.services.duck_pond
import DiveDB.services.utils.edf
importlib.reload(DiveDB.services.duck_pond)
importlib.reload(DiveDB.services.utils.edf)

from DiveDB.services.duck_pond import DuckPond
from DiveDB.services.utils.edf import create_mne_edf

duckpond = DuckPond()

conn = duckpond.get_delta_data(    
    animal_ids="mian-003",
    labels=["ECG_ICA2", "EEG_ICA5"],
    limit=1000000,
)

create_mne_edf(conn, "test.edf")

## Exporting Data to MNE Signal Array
For working with the data in MNE, we can export the data to an MNE Signal Array. This is useful for manipulating the data in MNE.

The `create_mne_array` function takes a DuckDB connection and returns an MNE RawArray.

##### Example:

In [ ]:
import importlib
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond
from DiveDB.services.utils.edf import create_mne_array

duckpond = DuckPond()

conn = duckpond.get_delta_data(    
    animal_ids="mian-003",
    labels="ECG_ICA2",
    limit=1000000,
)

raw = create_mne_array(conn, resample=100, l_freq=1, h_freq=20)
display(raw)